In [1]:
!which python
import pandas as pd
print(pd.__version__)
import sys
print(sys.version)
import inspect
print(inspect.getfile(pd))

/gpfs/bbp.cscs.ch/ssd/apps/bsd/2024-02-01/stage_externals/install_gcc-12.3.0-skylake/python-3.11.6-bj4i6m/bin/python
2.2.1
3.12.2 | packaged by Anaconda, Inc. | (main, Feb 27 2024, 17:35:02) [GCC 11.2.0]
/gpfs/bbp.cscs.ch/data/project/proj84/csaba/git/cellden/lib/python3.12/site-packages/pandas/__init__.py


**MAKE SURE THE PICKLE FILE USED HERE IS UP TO DATE!**
(It won't load with Python 3.9 or older.. )

The pickle file is created in density3_test_total_neuron_count_pickle.ipynb

The current validation script is here: 
- https://bbpgitlab.epfl.ch/molsys/personal/gene_to_ccfv3/-/blob/main/cell_atlas/validation.py?ref_type=heads

Currently it is checking for **cell, neuron, glia, astrocyte, microglia, oligodendrocyte, gad, pv, sst, vip, generic_excitatory, generic_inhibitory** numbers

In the **cerebellum, isocortex, fiber_tracts_ids, hippocampus, hippocampal_formation, thalamus, striatum, VPL, LGd, VPM, MOB** regions

Files to create:
   - .load_nrrd(cell_density)
   - neuron_density.nrrd
   - glia_density.nrrd
   - astrocyte_density.nrrd
   - microglia_density.nrrd
   - oligodendrocyte_density.nrrd
   - gad67+_density.nrrd
   - pv+_density.nrrd
   - sst+_density.nrrd
   - vip+_density.nrrd
   - Generic_Excitatory_Neuron_MType_Generic_Excitatory_Neuron_EType.nrrd
   - Generic_Inhibitory_Neuron_MType_Generic_Inhibitory_Neuron_EType.nrrd

Reconciling region ids:
- parcellation_term_label = 1.json / AllenCCF-Annotation-2020-
- parcellation_label = AllenCCF-Annotation-2020

In [2]:
import pickle
import pandas as pd
import json
from voxcell import RegionMap
# import nrrd
import numpy as np
import os
import copy

In [3]:
#This comes from on of the density1.5_test_total_neuron_count
path = '/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/'


# Load the dictionary of DataFrames from the pickle file
with open(f'{path}scaled_densities_new.pickle', 'rb') as f:
    total_cells = pickle.load(f)

# # Load the dictionary of DataFrames from the pickle file
# with open(f'{path}non_scaled_densities_new.pickle', 'rb') as f:
#     total_cells = pickle.load(f)

# # Load the dictionary of DataFrames from the pickle file
# with open(f'{path}scaled_densities_global_only.pickle', 'rb') as f:
#     total_cells = pickle.load(f)

# Check if 'UVU' exists
if 'UVU' not in total_cells:
    print("Non-leaf regions of the Cerebellum were removed")
else:
    print("'UVU' key exists in combined_result_dataframes")

Non-leaf regions of the Cerebellum were removed


In [4]:
#Assign these to regions above!
#total_cells.keys()


In [5]:
#region_map_path ='/gpfs/bbp.cscs.ch/project/proj62/csaba/atlas/bbp_prod_files/1.json'
region_map_path ='/gpfs/bbp.cscs.ch/data/project/proj84/atlas_pipeline_runs/2024-05-15T22:44:26+02:00/hierarchy_ccfv3_l23split_barrelsplit.json'
region_map = RegionMap.load_json(region_map_path)


In [6]:
file = '/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/metadata/parcellation_to_parcellation_term_membership_extend.csv'
parcellation_annotation = pd.read_csv(file)
#parcellation_annotation = parcellation_annotation[parcellation_annotation['parcellation_term_set_name'] == 'substructure']
print(parcellation_annotation.shape)

(3489, 19)


In [7]:
regions = list(total_cells.keys())
print(f'Regions _covered_ in the brain / total cells dict of df: {len(regions)}')
# Assuming 'my_list' is your list
if 'unassigned' in regions:
    print("Region was found, will remove: unassigned.")
    regions.remove('unassigned')
    print("For now we are removing the region unassigned as we cannot place it in the hierarchy.json system")
    
if 'brainunassigned' in regions:
    print("Region was found, will remove brain-unassigned.")
    regions.remove('brainunassigned')
    print("For now we are removing the region brain-unassigned as we cannot place it in the hierarchy.json system")    

if 'brain-unassigned' in regions:
    print("Region was found, will remove brain-unassigned.")
    regions.remove('brain-unassigned')
    print("For now we are removing the region brain-unassigned as we cannot place it in the hierarchy.json system") 


print(f"Remaining regions: {len(regions)}")
region_info = parcellation_annotation[parcellation_annotation['cluster_as_filename'].isin(regions)]
region_info.shape

Regions _covered_ in the brain / total cells dict of df: 703
Region was found, will remove: unassigned.
For now we are removing the region unassigned as we cannot place it in the hierarchy.json system
Region was found, will remove brain-unassigned.
For now we are removing the region brain-unassigned as we cannot place it in the hierarchy.json system
Remaining regions: 701


(959, 19)

In [11]:
missing_regions = parcellation_annotation[~parcellation_annotation['parcellation_term_acronym'].isin(regions)]
missin_substructures = missing_regions[missing_regions['parcellation_term_set_name'] == 'substructure']
print(f"All substructures not covered by merfish slices: \n1. {missin_substructures['parcellation_term_name'].values}")

All substructures not covered by merfish slices: 
1. ['Frontal pole, layer 1' 'Frontal pole, layer 2/3'
 'Primary motor area, Layer 2/3' 'Secondary motor area, layer 2/3'
 'Primary somatosensory area, nose, layer 1'
 'Primary somatosensory area, nose, layer 2/3'
 'Primary somatosensory area, nose, layer 4'
 'Primary somatosensory area, nose, layer 5'
 'Primary somatosensory area, nose, layer 6a'
 'Primary somatosensory area, nose, layer 6b'
 'Primary somatosensory area, barrel field, layer 1'
 'Primary somatosensory area, barrel field, layer 2/3'
 'Primary somatosensory area, barrel field, layer 4'
 'Primary somatosensory area, barrel field, layer 5'
 'Primary somatosensory area, barrel field, layer 6a'
 'Primary somatosensory area, barrel field, layer 6b'
 'Primary somatosensory area, lower limb, layer 1'
 'Primary somatosensory area, lower limb, layer 2/3'
 'Primary somatosensory area, lower limb, layer 4'
 'Primary somatosensory area, lower limb, layer 5'
 'Primary somatosensory are

In [34]:
#If this does not trigger every region acronym is a substructure, if it triggers, the region is either not a substructure or have higher than substructure term set name
for region in regions:
    #acronym = region_info[region_info['parcellation_term_acronym'] == region]
    acronym = region_info[region_info['cluster_as_filename'] == region]
    if acronym.shape[0] == 0:
        print(f"{region} is not only a substructure! ")

region_info.shape

(959, 19)

In [35]:
# Identify duplicates in parcellation_term_acronym
duplicates = region_info[region_info.duplicated('parcellation_term_acronym', keep=False)]

# Filter to keep only rows marked as 'substructure'
substructure_filter = duplicates['parcellation_term_set_name'] == 'substructure'

# Keep substructure duplicates and non-duplicated rows
filtered_region_info = pd.concat([region_info[~region_info['parcellation_term_acronym'].isin(duplicates['parcellation_term_acronym'])], duplicates[substructure_filter]])

# Reset the index (optional, for cleaner output)
filtered_region_info = filtered_region_info.reset_index(drop=True)

filtered_region_info.shape

(708, 19)

## Region parcellation based on validation code

We can check which region is covered by the current validation code __registered_regions__ and which are not __missing_reg__. 
Strings to check: regions where literature data is available
Second list: remaining areas in the brain. 


In [36]:
# List of strings to check
strings_to_check = ['Cerebellum', 'Isocortex', 'fiber tracts', 'ventricular systems', 'Hippocampus', 
                    'Hippocampal formation', 'Thalamus', 'Striatum', 'VPL', 'LGd', 'VPM', 'MOB']

second_list = ['Brain stem', 'Olfactory areas', 'Cortical subplate', 'Cerebral nuclei', ]
missing_reg = []
registered_regions = []

for region in regions:
    #print(region)
    acronym = filtered_region_info[filtered_region_info['cluster_as_filename'] == region][['parcellation_term_acronym', 'parcellation_term_name', 'parcellation_label']]
    long_id = filtered_region_info[filtered_region_info['cluster_as_filename'] == region]['parcellation_label']
    substring = "AllenCCF-Annotation-2020-"
    index = long_id.iloc[0].find(substring)
    id = long_id.iloc[0][index + len(substring):]
    # print(long_id.iloc[0], id)
    ascendants = region_map.get(int(id), "name", with_ascendants=True)
    # Assuming ascendant is your list
    found_strings = [s for s in strings_to_check if any(s in a for a in ascendants)]
    found_strings_2 = [s for s in second_list if any(s in a for a in ascendants)]
    if found_strings:
        #print(f"Found strings in the ascendant list: {found_strings}")
        registered_regions.append(id)
    elif found_strings_2:
        missing_reg.append(id)
        #print(f"Found strings in second list: {found_strings_2}")
    else:
        print(id)
        print("One region can not be placed in the brain.")
        break

In [37]:
len(registered_regions), len(missing_reg)

(473, 228)

## Calculate total cells and density in a region: Isocortex

Deal with regions missing from the total cells

In [38]:
isocortex = (
    region_map.find("Isocortex", attr="name", with_descendants=True)
    | region_map.find("Entorhinal area", attr="name", with_descendants=True)
    | region_map.find("Piriform area", attr="name", with_descendants=True)
)
cerebellum = region_map.find(
        "Cerebellum", attr="name", with_descendants=True
    ) | region_map.find("arbor vitae", attr="name", with_descendants=True)

fiber_tracts_ids = (
    region_map.find("fiber tracts", attr="name", with_descendants=True)
    | region_map.find("grooves", attr="name", with_descendants=True)
    | region_map.find("ventricular systems", attr="name", with_descendants=True)
)
hippocampus = (
    region_map.find("Hippocampal region", attr="name", with_descendants=True)
)
hippocampal_formation = (
    region_map.find("Hippocampal formation", attr="name", with_descendants=True)
)
thalamus = (
    region_map.find("Thalamus", attr="name", with_descendants=True)
)
striatum = (
    region_map.find("Striatum", attr="name", with_descendants=True)
)
VPL = (
    region_map.find("Ventral posterolateral nucleus of the thalamus", attr="name", with_descendants=True)
)
LGd = (
    region_map.find("Dorsal part of the lateral geniculate complex", attr="name", with_descendants=True)
)
VPM = (
    region_map.find("Ventral posteromedial nucleus of the thalamus", attr="name", with_descendants=True)
)
MOB = (
    region_map.find("Main olfactory bulb", attr="name", with_descendants=True)
)

print("Done!")

Done!


In [39]:
#This cell will complement the two cells above
Hindbrain = (
    region_map.find("Hindbrain", attr="name", with_descendants=True)
)
Midbrain = (
    region_map.find("Midbrain", attr="name", with_descendants=True)
)
Interbrain = (
    region_map.find("Interbrain", attr="name", with_descendants=True)
)
Olfactoryareas = (
    region_map.find("Olfactory areas", attr="name", with_descendants=True)
)
Corticalsubplate = (
    region_map.find("Cortical subplate", attr="name", with_descendants=True)
)
Cerebralnuclei = (
    region_map.find("Cerebral nuclei", attr="name", with_descendants=True)
)
cerebral_cortex = (
    region_map.find("Cerebral cortex", attr="name", with_descendants=True)
)
wholebrain = (
    region_map.find("root", attr="name", with_descendants=True)
)

#This is the final dictionary of all region names and their region ids
areas = {
    'wholebrain': wholebrain, 'Cerebral cortex': cerebral_cortex,
    'isocortex': isocortex, 'cerebellum': cerebellum, 'fiber_tracts_ids': fiber_tracts_ids, 
    'hippocampus': hippocampus, 'hippocampal_formation':hippocampal_formation, 
    'thalamus': thalamus, 'striatum': striatum, 'VPL': VPL, 'LGd': LGd, 'VPM': VPM, 'MOB': MOB,
    'Olfactory areas': Olfactoryareas, 'Cortical subplate': Corticalsubplate, 
    'Cerebral nuclei': Cerebralnuclei, 'Interbrain': Interbrain , 'Midbrain': Midbrain, 
    'Interbrain': Interbrain
}

In [40]:
total_cells_copy = copy.deepcopy(total_cells) 
#Here there is no filter applied from above, so it is total cells

for key, value in areas.items():
    
    keys_to_drop = ['unassigned', 'brain-unassigned']
    for region in regions:
        
        filtered_data = region_info[(region_info['cluster_as_filename'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        id_ = filtered_data['label_numbers'].iloc[0]
        
        if int(id_) in value:
            #print(region, id_, "is in the set")
            None
        else:
            #print(id_, "is not in the set")
            keys_to_drop.append(region)

        #break    

    # Create a new dictionary without the specified keys (all leaf regions and their cell types in 1 region)
    total_cells_filtered = {key: value for key, value in total_cells_copy.items() if key not in keys_to_drop}

    # Concatenate all DataFrames into a single DataFrame
    concatenated_df = pd.concat(total_cells_filtered.values())

    # Group by all columns except the 'total_cells' column and sum the values
    summed_df = concatenated_df.groupby(concatenated_df.index).sum()
    print(f"The brain area {key} has {summed_df.shape[0]} different cell types")
    print("Total sum of cells:", '{:,.0f}'.format(summed_df['density_mm3'].sum()))

The brain area wholebrain has 5273 different cell types
Total sum of cells: 142,616,594
The brain area Cerebral cortex has 1902 different cell types
Total sum of cells: 46,572,026
The brain area isocortex has 1538 different cell types
Total sum of cells: 36,001,343
The brain area cerebellum has 1205 different cell types
Total sum of cells: 24,747,202
The brain area fiber_tracts_ids has 3333 different cell types
Total sum of cells: 18,573,856
The brain area hippocampus has 1122 different cell types
Total sum of cells: 3,884,649
The brain area hippocampal_formation has 1364 different cell types
Total sum of cells: 6,246,210
The brain area thalamus has 1245 different cell types
Total sum of cells: 9,976,464
The brain area striatum has 1927 different cell types
Total sum of cells: 2,940,098
The brain area VPL has 845 different cell types
Total sum of cells: 293,537
The brain area LGd has 829 different cell types
Total sum of cells: 757,948
The brain area VPM has 833 different cell types
To

In [41]:
meta_path = "/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/metadata/WMB-10X/20231215/views/cell_metadata_with_cluster_annotation.csv"
metadata = pd.read_csv(meta_path, dtype={'cell_label':str})
metadata = metadata[['class', 'subclass', 'cluster']]

n_classes = ['01 IT-ET Glut', '02 NP-CT-L6b Glut', '03 OB-CR Glut',
       '04 DG-IMN Glut', '05 OB-IMN GABA', '06 CTX-CGE GABA',
       '07 CTX-MGE GABA', '08 CNU-MGE GABA', '09 CNU-LGE GABA',
       '10 LSX GABA', '11 CNU-HYa GABA', '12 HY GABA', '13 CNU-HYa Glut',
       '14 HY Glut', '15 HY Gnrh1 Glut', '16 HY MM Glut', '17 MH-LH Glut',
       '18 TH Glut', '19 MB Glut', '20 MB GABA', '21 MB Dopa',
       '22 MB-HB Sero', '23 P Glut', '24 MY Glut', '25 Pineal Glut',
       '26 P GABA', '27 MY GABA', '28 CB GABA', '29 CB Glut',]

nn_classes = ['30 Astro-Epen', '31 OPC-Oligo', '32 OEC', '33 Vascular',
       '34 Immune']

exc = ['01 IT-ET Glut', '02 NP-CT-L6b Glut', '03 OB-CR Glut',
      '04 DG-IMN Glut', '13 CNU-HYa Glut', '14 HY Glut', '15 HY Gnrh1 Glut', '16 HY MM Glut', '17 MH-LH Glut',
      '18 TH Glut', '19 MB Glut', '23 P Glut', '24 MY Glut', '25 Pineal Glut', '29 CB Glut',]
inh = ['05 OB-IMN GABA', '06 CTX-CGE GABA', '07 CTX-MGE GABA', '08 CNU-MGE GABA', '09 CNU-LGE GABA',
      '10 LSX GABA', '11 CNU-HYa GABA', '12 HY GABA', '20 MB GABA', '26 P GABA', '27 MY GABA', '28 CB GABA', ]
other = ['21 MB Dopa', '22 MB-HB Sero', ]
exci_inhib_su = exc + inh

astrotypes = ['5206 Bergmann NN_1', '5207 Astro-CB NN_1', '5208 Astro-NT NN_1',
       '5209 Astro-NT NN_1', '5210 Astro-NT NN_1', '5211 Astro-NT NN_1',
       '5212 Astro-NT NN_1', '5213 Astro-NT NN_1', '5214 Astro-NT NN_2',
       '5215 Astro-NT NN_2', '5216 Astro-NT NN_2', '5217 Astro-NT NN_2',
       '5218 Astro-TE NN_1', '5219 Astro-TE NN_1', '5220 Astro-TE NN_1',
       '5221 Astro-TE NN_1', '5222 Astro-TE NN_2', '5223 Astro-TE NN_2',
       '5224 Astro-TE NN_3', '5225 Astro-TE NN_3', '5226 Astro-TE NN_3',
       '5227 Astro-TE NN_3', '5228 Astro-TE NN_4', '5229 Astro-TE NN_5',
       '5230 Astro-TE NN_5', '5231 Astro-OLF NN_1', '5232 Astro-OLF NN_1',
       '5233 Astro-OLF NN_2', '5234 Astro-OLF NN_2',
       '5235 Astro-OLF NN_3', '5236 Astro-OLF NN_3',]

microglia = ['5312 Microglia NN_1']

oligos = ['5266 OPC NN_1', '5267 OPC NN_1',
       '5268 OPC NN_1', '5269 OPC NN_1', '5270 OPC NN_1', '5271 OPC NN_2',
       '5272 COP NN_1', '5273 COP NN_1', '5274 COP NN_1', '5275 COP NN_1',
       '5276 COP NN_1', '5277 COP NN_1', '5278 NFOL NN_2',
       '5279 NFOL NN_2', '5280 NFOL NN_2', '5281 NFOL NN_2',
       '5282 MFOL NN_3', '5283 MFOL NN_3', '5284 MOL NN_4',
       '5285 MOL NN_4', '5286 MOL NN_4', '5287 MOL NN_4', '5288 MOL NN_4',]

glia = astrotypes + microglia + oligos

neurontypes = np.unique(metadata[metadata['class'].isin(n_classes)]['cluster'].values)
nonneurontypes = np.unique(metadata[metadata['class'].isin(nn_classes)]['cluster'].values)
exctypes = np.unique(metadata[metadata['class'].isin(exc)]['cluster'].values)
inhtypes = np.unique(metadata[metadata['class'].isin(inh)]['cluster'].values)
othertypes = np.unique(metadata[metadata['class'].isin(other)]['cluster'].values)
exci_inhib_sum = np.unique(metadata[metadata['class'].isin(exci_inhib_su)]['cluster'].values)

/nvme/veraszto/2720273/ipykernel_98654/1153689541.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(meta_path, dtype={'cell_label':str})


In [42]:
total_cells_copy = copy.deepcopy(total_cells) 

for key, df in total_cells_copy.items():
    total_cells_copy[key] = df[df.index.isin(neurontypes)]

for key, value in areas.items():
    
    keys_to_drop = ['unassigned', 'brain-unassigned']
    for region in regions:
        
        filtered_data = region_info[(region_info['cluster_as_filename'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        id_ = filtered_data['label_numbers'].iloc[0]
        
        if int(id_) in value:
            #print(region, id_, "is in the set")
            None
        else:
            #print(id_, "is not in the set")
            keys_to_drop.append(region)

        #break    

    # Create a new dictionary without the specified keys
    total_cells_filtered = {key: value for key, value in total_cells_copy.items() if key not in keys_to_drop}

    # Concatenate all DataFrames into a single DataFrame
    concatenated_df = pd.concat(total_cells_filtered.values())

    # Group by all columns except the 'total_cells' column and sum the values
    summed_df = concatenated_df.groupby(concatenated_df.index).sum()
    print(f"The brain area {key} has {summed_df.shape[0]} different neuron types")
    print("Total sum of these cells:", '{:,.0f}'.format(summed_df['density_mm3'].sum()))

The brain area wholebrain has 5156 different neuron types
Total sum of these cells: 70,801,927
The brain area Cerebral cortex has 1800 different neuron types
Total sum of these cells: 22,042,946
The brain area isocortex has 1444 different neuron types
Total sum of these cells: 16,666,247
The brain area cerebellum has 1116 different neuron types
Total sum of these cells: 22,044,942
The brain area fiber_tracts_ids has 3218 different neuron types
Total sum of these cells: 4,066,982
The brain area hippocampus has 1030 different neuron types
Total sum of these cells: 1,859,631
The brain area hippocampal_formation has 1271 different neuron types
Total sum of these cells: 2,709,141
The brain area thalamus has 1157 different neuron types
Total sum of these cells: 5,806,367
The brain area striatum has 1834 different neuron types
Total sum of these cells: 1,260,683
The brain area VPL has 763 different neuron types
Total sum of these cells: 79,000
The brain area LGd has 747 different neuron types

In [43]:
total_cells_copy = copy.deepcopy(total_cells) 

for key, df in total_cells_copy.items():
    total_cells_copy[key] = df[df.index.isin(nonneurontypes)]

for key, value in areas.items():
    
    keys_to_drop = ['unassigned', 'brain-unassigned']
    for region in regions:
        
        filtered_data = region_info[(region_info['cluster_as_filename'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        id_ = filtered_data['label_numbers'].iloc[0]
        
        if int(id_) in value:
            #print(region, id_, "is in the set")
            None
        else:
            #print(id_, "is not in the set")
            keys_to_drop.append(region)

        #break    

    # Create a new dictionary without the specified keys
    total_cells_filtered = {key: value for key, value in total_cells_copy.items() if key not in keys_to_drop}

    # Concatenate all DataFrames into a single DataFrame
    concatenated_df = pd.concat(total_cells_filtered.values())

    # Group by all columns except the 'total_cells' column and sum the values
    summed_df = concatenated_df.groupby(concatenated_df.index).sum()
    print(f"The brain area {key} has {summed_df.shape[0]}/{len(nonneurontypes)} different nonneuron types")
    print("Total sum of these cells:", '{:,.0f}'.format(summed_df['density_mm3'].sum()))

The brain area wholebrain has 117/117 different nonneuron types
Total sum of these cells: 71,814,667
The brain area Cerebral cortex has 102/117 different nonneuron types
Total sum of these cells: 24,529,080
The brain area isocortex has 94/117 different nonneuron types
Total sum of these cells: 19,335,096
The brain area cerebellum has 89/117 different nonneuron types
Total sum of these cells: 2,702,260
The brain area fiber_tracts_ids has 115/117 different nonneuron types
Total sum of these cells: 14,506,875
The brain area hippocampus has 92/117 different nonneuron types
Total sum of these cells: 2,025,018
The brain area hippocampal_formation has 93/117 different nonneuron types
Total sum of these cells: 3,537,069
The brain area thalamus has 88/117 different nonneuron types
Total sum of these cells: 4,170,097
The brain area striatum has 93/117 different nonneuron types
Total sum of these cells: 1,679,415
The brain area VPL has 82/117 different nonneuron types
Total sum of these cells: 21

In [44]:
total_cells_copy = copy.deepcopy(total_cells) 

for key, df in total_cells_copy.items():
    total_cells_copy[key] = df[df.index.isin(exctypes)]

for key, value in areas.items():
    
    keys_to_drop = ['unassigned', 'brain-unassigned']
    for region in regions:
        
        filtered_data = region_info[(region_info['cluster_as_filename'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        id_ = filtered_data['label_numbers'].iloc[0]
        
        if int(id_) in value:
            #print(region, id_, "is in the set")
            None
        else:
            #print(id_, "is not in the set")
            keys_to_drop.append(region)

        #break    

    # Create a new dictionary without the specified keys
    total_cells_filtered = {key: value for key, value in total_cells_copy.items() if key not in keys_to_drop}

    # Concatenate all DataFrames into a single DataFrame
    concatenated_df = pd.concat(total_cells_filtered.values())

    # Group by all columns except the 'total_cells' column and sum the values
    summed_df = concatenated_df.groupby(concatenated_df.index).sum()
    print(f"The brain area {key} has {summed_df.shape[0]} different EXC neuron types")
    print("Total sum of these cells:", '{:,.0f}'.format(summed_df['density_mm3'].sum()))

The brain area wholebrain has 2582 different EXC neuron types
Total sum of these cells: 59,949,487
The brain area Cerebral cortex has 875 different EXC neuron types
Total sum of these cells: 19,201,771
The brain area isocortex has 727 different EXC neuron types
Total sum of these cells: 15,812,335
The brain area cerebellum has 567 different EXC neuron types
Total sum of these cells: 21,491,282
The brain area fiber_tracts_ids has 1493 different EXC neuron types
Total sum of these cells: 3,063,101
The brain area hippocampus has 475 different EXC neuron types
Total sum of these cells: 1,718,376
The brain area hippocampal_formation has 663 different EXC neuron types
Total sum of these cells: 2,506,235
The brain area thalamus has 587 different EXC neuron types
Total sum of these cells: 5,603,076
The brain area striatum has 653 different EXC neuron types
Total sum of these cells: 229,559
The brain area VPL has 337 different EXC neuron types
Total sum of these cells: 72,259
The brain area LGd

In [45]:
total_cells_copy = copy.deepcopy(total_cells) 

for key, df in total_cells_copy.items():
    total_cells_copy[key] = df[df.index.isin(inhtypes)]

for key, value in areas.items():
    
    keys_to_drop = ['unassigned', 'brain-unassigned']
    for region in regions:
        
        filtered_data = region_info[(region_info['cluster_as_filename'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        id_ = filtered_data['label_numbers'].iloc[0]
        
        if int(id_) in value:
            #print(region, id_, "is in the set")
            None
        else:
            #print(id_, "is not in the set")
            keys_to_drop.append(region)

        #break    

    # Create a new dictionary without the specified keys
    total_cells_filtered = {key: value for key, value in total_cells_copy.items() if key not in keys_to_drop}

    # Concatenate all DataFrames into a single DataFrame
    concatenated_df = pd.concat(total_cells_filtered.values())

    # Group by all columns except the 'total_cells' column and sum the values
    summed_df = concatenated_df.groupby(concatenated_df.index).sum()
    print(f"The brain area {key} has {summed_df.shape[0]} different INH neuron types")
    print("Total sum of these cells:", '{:,.0f}'.format(summed_df['density_mm3'].sum()))

The brain area wholebrain has 2499 different INH neuron types
Total sum of these cells: 10,016,141
The brain area Cerebral cortex has 923 different INH neuron types
Total sum of these cells: 2,841,160
The brain area isocortex has 715 different INH neuron types
Total sum of these cells: 853,898
The brain area cerebellum has 547 different INH neuron types
Total sum of these cells: 553,646
The brain area fiber_tracts_ids has 1663 different INH neuron types
Total sum of these cells: 892,265
The brain area hippocampus has 553 different INH neuron types
Total sum of these cells: 141,240
The brain area hippocampal_formation has 606 different INH neuron types
Total sum of these cells: 202,892
The brain area thalamus has 563 different INH neuron types
Total sum of these cells: 202,117
The brain area striatum has 1179 different INH neuron types
Total sum of these cells: 1,031,110
The brain area VPL has 424 different INH neuron types
Total sum of these cells: 6,727
The brain area LGd has 412 diff

In [46]:
total_cells_copy = copy.deepcopy(total_cells) 

for key, df in total_cells_copy.items():
    total_cells_copy[key] = df[df.index.isin(othertypes)]

for key, value in areas.items():
    
    keys_to_drop = ['unassigned', 'brain-unassigned']
    for region in regions:
        
        filtered_data = region_info[(region_info['cluster_as_filename'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        id_ = filtered_data['label_numbers'].iloc[0]
        
        if int(id_) in value:
            #print(region, id_, "is in the set")
            None
        else:
            #print(id_, "is not in the set")
            keys_to_drop.append(region)

        #break    

    # Create a new dictionary without the specified keys
    total_cells_filtered = {key: value for key, value in total_cells_copy.items() if key not in keys_to_drop}

    # Concatenate all DataFrames into a single DataFrame
    concatenated_df = pd.concat(total_cells_filtered.values())

    # Group by all columns except the 'total_cells' column and sum the values
    summed_df = concatenated_df.groupby(concatenated_df.index).sum()
    print(f"The brain area {key} has {summed_df.shape[0]} different OTHER neuron types")
    print("Total sum of these cells:", '{:,.0f}'.format(summed_df['density_mm3'].sum()))

The brain area wholebrain has 75 different OTHER neuron types
Total sum of these cells: 836,299
The brain area Cerebral cortex has 2 different OTHER neuron types
Total sum of these cells: 14
The brain area isocortex has 2 different OTHER neuron types
Total sum of these cells: 14
The brain area cerebellum has 2 different OTHER neuron types
Total sum of these cells: 14
The brain area fiber_tracts_ids has 62 different OTHER neuron types
Total sum of these cells: 111,615
The brain area hippocampus has 2 different OTHER neuron types
Total sum of these cells: 14
The brain area hippocampal_formation has 2 different OTHER neuron types
Total sum of these cells: 14
The brain area thalamus has 7 different OTHER neuron types
Total sum of these cells: 1,174
The brain area striatum has 2 different OTHER neuron types
Total sum of these cells: 14
The brain area VPL has 2 different OTHER neuron types
Total sum of these cells: 14
The brain area LGd has 2 different OTHER neuron types
Total sum of these c

In [47]:
#glia
total_cells_copy = copy.deepcopy(total_cells) 

for key, df in total_cells_copy.items():
    total_cells_copy[key] = df[df.index.isin(glia)]

for key, value in areas.items():
    
    keys_to_drop = ['unassigned', 'brain-unassigned']
    for region in regions:
        
        filtered_data = region_info[(region_info['cluster_as_filename'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        id_ = filtered_data['label_numbers'].iloc[0]
        
        if int(id_) in value:
            #print(region, id_, "is in the set")
            None
        else:
            #print(id_, "is not in the set")
            keys_to_drop.append(region)

        #break    

    # Create a new dictionary without the specified keys
    total_cells_filtered = {key: value for key, value in total_cells_copy.items() if key not in keys_to_drop}

    # Concatenate all DataFrames into a single DataFrame
    concatenated_df = pd.concat(total_cells_filtered.values())

    # Group by all columns except the 'total_cells' column and sum the values
    summed_df = concatenated_df.groupby(concatenated_df.index).sum()
    print(f"The brain area {key} has {summed_df.shape[0]} different GLIA types")
    print("Total sum of these cells:", '{:,.0f}'.format(summed_df['density_mm3'].sum()))

The brain area wholebrain has 55 different GLIA types
Total sum of these cells: 54,123,310
The brain area Cerebral cortex has 53 different GLIA types
Total sum of these cells: 17,759,667
The brain area isocortex has 49 different GLIA types
Total sum of these cells: 13,953,535
The brain area cerebellum has 42 different GLIA types
Total sum of these cells: 1,807,764
The brain area fiber_tracts_ids has 53 different GLIA types
Total sum of these cells: 11,226,297
The brain area hippocampus has 47 different GLIA types
Total sum of these cells: 1,565,288
The brain area hippocampal_formation has 48 different GLIA types
Total sum of these cells: 2,675,894
The brain area thalamus has 42 different GLIA types
Total sum of these cells: 3,190,145
The brain area striatum has 48 different GLIA types
Total sum of these cells: 1,257,458
The brain area VPL has 39 different GLIA types
Total sum of these cells: 148,255
The brain area LGd has 39 different GLIA types
Total sum of these cells: 219,847
The br

In [48]:
#astrotypes
total_cells_copy = copy.deepcopy(total_cells) 

for key, df in total_cells_copy.items():
    total_cells_copy[key] = df[df.index.isin(astrotypes)]

for key, value in areas.items():
    
    keys_to_drop = ['unassigned', 'brain-unassigned']
    for region in regions:
        
        filtered_data = region_info[(region_info['cluster_as_filename'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        id_ = filtered_data['label_numbers'].iloc[0]
        
        if int(id_) in value:
            #print(region, id_, "is in the set")
            None
        else:
            #print(id_, "is not in the set")
            keys_to_drop.append(region)

        #break    

    # Create a new dictionary without the specified keys
    total_cells_filtered = {key: value for key, value in total_cells_copy.items() if key not in keys_to_drop}

    # Concatenate all DataFrames into a single DataFrame
    concatenated_df = pd.concat(total_cells_filtered.values())

    # Group by all columns except the 'total_cells' column and sum the values
    summed_df = concatenated_df.groupby(concatenated_df.index).sum()
    print(f"The brain area {key} has {summed_df.shape[0]} different astrocyte types")
    print("Total sum of these cells:", '{:,.0f}'.format(summed_df['density_mm3'].sum()))

The brain area wholebrain has 31 different astrocyte types
Total sum of these cells: 25,164,478
The brain area Cerebral cortex has 29 different astrocyte types
Total sum of these cells: 10,516,280
The brain area isocortex has 26 different astrocyte types
Total sum of these cells: 7,907,544
The brain area cerebellum has 20 different astrocyte types
Total sum of these cells: 961,301
The brain area fiber_tracts_ids has 29 different astrocyte types
Total sum of these cells: 3,610,196
The brain area hippocampus has 24 different astrocyte types
Total sum of these cells: 1,113,771
The brain area hippocampal_formation has 24 different astrocyte types
Total sum of these cells: 1,867,524
The brain area thalamus has 21 different astrocyte types
Total sum of these cells: 1,466,061
The brain area striatum has 24 different astrocyte types
Total sum of these cells: 730,410
The brain area VPL has 18 different astrocyte types
Total sum of these cells: 56,608
The brain area LGd has 18 different astrocyt

In [49]:
#microglia
total_cells_copy = copy.deepcopy(total_cells) 

for key, df in total_cells_copy.items():
    total_cells_copy[key] = df[df.index.isin(microglia)]

for key, value in areas.items():
    
    keys_to_drop = ['unassigned', 'brain-unassigned']
    for region in regions:
        
        filtered_data = region_info[(region_info['cluster_as_filename'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        id_ = filtered_data['label_numbers'].iloc[0]
        
        if int(id_) in value:
            #print(region, id_, "is in the set")
            None
        else:
            #print(id_, "is not in the set")
            keys_to_drop.append(region)

        #break    

    # Create a new dictionary without the specified keys
    total_cells_filtered = {key: value for key, value in total_cells_copy.items() if key not in keys_to_drop}

    # Concatenate all DataFrames into a single DataFrame
    concatenated_df = pd.concat(total_cells_filtered.values())

    # Group by all columns except the 'total_cells' column and sum the values
    summed_df = concatenated_df.groupby(concatenated_df.index).sum()
    print(f"The brain area {key} has {summed_df.shape[0]} different microglia types")
    print("Total sum of these cells:", '{:,.0f}'.format(summed_df['density_mm3'].sum()))

The brain area wholebrain has 1 different microglia types
Total sum of these cells: 2,265,089
The brain area Cerebral cortex has 1 different microglia types
Total sum of these cells: 1,265,702
The brain area isocortex has 1 different microglia types
Total sum of these cells: 1,059,558
The brain area cerebellum has 1 different microglia types
Total sum of these cells: 51,838
The brain area fiber_tracts_ids has 1 different microglia types
Total sum of these cells: 291,776
The brain area hippocampus has 1 different microglia types
Total sum of these cells: 72,009
The brain area hippocampal_formation has 1 different microglia types
Total sum of these cells: 128,998
The brain area thalamus has 1 different microglia types
Total sum of these cells: 92,408
The brain area striatum has 1 different microglia types
Total sum of these cells: 59,850
The brain area VPL has 1 different microglia types
Total sum of these cells: 2,970
The brain area LGd has 1 different microglia types
Total sum of these

In [50]:
#oligos
total_cells_copy = copy.deepcopy(total_cells) 

for key, df in total_cells_copy.items():
    total_cells_copy[key] = df[df.index.isin(oligos)]

for key, value in areas.items():
    
    keys_to_drop = ['unassigned', 'brain-unassigned']
    for region in regions:
        
        filtered_data = region_info[(region_info['cluster_as_filename'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        id_ = filtered_data['label_numbers'].iloc[0]
        
        if int(id_) in value:
            #print(region, id_, "is in the set")
            None
        else:
            #print(id_, "is not in the set")
            keys_to_drop.append(region)

        #break    

    # Create a new dictionary without the specified keys
    total_cells_filtered = {key: value for key, value in total_cells_copy.items() if key not in keys_to_drop}

    # Concatenate all DataFrames into a single DataFrame
    concatenated_df = pd.concat(total_cells_filtered.values())

    # Group by all columns except the 'total_cells' column and sum the values
    summed_df = concatenated_df.groupby(concatenated_df.index).sum()
    print(f"The brain area {key} has {summed_df.shape[0]} different oligocyte types")
    print("Total sum of these cells:", '{:,.0f}'.format(summed_df['density_mm3'].sum()))

The brain area wholebrain has 23 different oligocyte types
Total sum of these cells: 26,693,743
The brain area Cerebral cortex has 23 different oligocyte types
Total sum of these cells: 5,977,685
The brain area isocortex has 22 different oligocyte types
Total sum of these cells: 4,986,433
The brain area cerebellum has 21 different oligocyte types
Total sum of these cells: 794,625
The brain area fiber_tracts_ids has 23 different oligocyte types
Total sum of these cells: 7,324,325
The brain area hippocampus has 22 different oligocyte types
Total sum of these cells: 379,508
The brain area hippocampal_formation has 23 different oligocyte types
Total sum of these cells: 679,371
The brain area thalamus has 20 different oligocyte types
Total sum of these cells: 1,631,676
The brain area striatum has 23 different oligocyte types
Total sum of these cells: 467,199
The brain area VPL has 20 different oligocyte types
Total sum of these cells: 88,678
The brain area LGd has 20 different oligocyte typ

In [51]:
#exci_inhib_sum
total_cells_copy = copy.deepcopy(total_cells) 

for key, df in total_cells_copy.items():
    total_cells_copy[key] = df[df.index.isin(exci_inhib_sum)]

for key, value in areas.items():
    
    keys_to_drop = ['unassigned', 'brain-unassigned']
    for region in regions:
        
        filtered_data = region_info[(region_info['cluster_as_filename'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        id_ = filtered_data['label_numbers'].iloc[0]
        
        if int(id_) in value:
            #print(region, id_, "is in the set")
            None
        else:
            #print(id_, "is not in the set")
            keys_to_drop.append(region)

        #break    

    # Create a new dictionary without the specified keys
    total_cells_filtered = {key: value for key, value in total_cells_copy.items() if key not in keys_to_drop}

    # Concatenate all DataFrames into a single DataFrame
    concatenated_df = pd.concat(total_cells_filtered.values())

    # Group by all columns except the 'total_cells' column and sum the values
    summed_df = concatenated_df.groupby(concatenated_df.index).sum()
    print(f"The brain area {key} has {summed_df.shape[0]} different INH+EXC(Sum) types")
    print("Total sum of these cells:", '{:,.0f}'.format(summed_df['density_mm3'].sum()))

The brain area wholebrain has 5081 different INH+EXC(Sum) types
Total sum of these cells: 69,965,628
The brain area Cerebral cortex has 1798 different INH+EXC(Sum) types
Total sum of these cells: 22,042,931
The brain area isocortex has 1442 different INH+EXC(Sum) types
Total sum of these cells: 16,666,233
The brain area cerebellum has 1114 different INH+EXC(Sum) types
Total sum of these cells: 22,044,928
The brain area fiber_tracts_ids has 3156 different INH+EXC(Sum) types
Total sum of these cells: 3,955,366
The brain area hippocampus has 1028 different INH+EXC(Sum) types
Total sum of these cells: 1,859,617
The brain area hippocampal_formation has 1269 different INH+EXC(Sum) types
Total sum of these cells: 2,709,127
The brain area thalamus has 1150 different INH+EXC(Sum) types
Total sum of these cells: 5,805,193
The brain area striatum has 1832 different INH+EXC(Sum) types
Total sum of these cells: 1,260,669
The brain area VPL has 761 different INH+EXC(Sum) types
Total sum of these cel